# [部分単語のトークン化](https://www.tensorflow.org/tutorials/tensorflow_text/subwords_tokenizer)

データセットからサブワード語彙を生成し、それらを使って`text.BertTokenizer`を語彙から構築する。

サブワードトークナイザの主な利点は、単語ベースのトークン化と文字ベースのトークン化の間に加筆することができる。

In [1]:
import collections
import os
import pathlib
import re
import string
import sys
import tempfile
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [2]:
tf.get_logger().setLevel('ERROR')
pwd = pathlib.Path.cwd()

`tensorflow_text`は三つのサブワードスタイルのトークナイザを持っている。

- `text.BertTokenizer`: ハイレベルインターフェースのクラス。BERTのトークン分割アルゴリズムと`WordpiceTokenizer`を含む。文章を入力として受け取り、トークンIDを返す。
- `text.WordpieceTokenizer`: レベルの低いインターフェースのクラス。[WordPiece algorithm](https://www.tensorflow.org/tutorials/tensorflow_text/subwords_tokenizer#applying_wordpiece)のみ実装されている。呼び出す前にテキストを標準化して単語に分割する必要がある。入力として単語を受け取り、トークンIDを返す。
- `text.SentencepieceTokenizer`: より複雑な設定が必要。初期化メソッドは学習前の*sentencepiece*モデルを必要とする。構築のやり方は[sentencepiece repository](https://github.com/google/sentencepiece#train-sentencepiece-model)を参照。トークン化する際に、入力として文章を受け付ける。

ここでは、トップダウン方式で既存の単語群から*wordpiece vocabulary*を構築する。これは日本語、中国語、韓国語には適用できない（これらの言語のトークン化には`text.SentencepieceTokenizer`、`text.UnicodeCharTokenizer`、もしくは[このアプローチ](https://tfhub.dev/google/zh_segmentation/1)を使う）。

## データセットのダウンロード

ここではポルトガル語と英語の翻訳データセットをtfdsからダウンロードする。このデータセットは全て小文字で、句読点の周りにはスペースがあり、なんのユニコード標準化手法が使われているかは不明である。

In [3]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

In [4]:
for pt, en in train_examples.take(1):
    print("Portuguese: ", pt.numpy().decode('utf-8'))
    print("English:   ", en.numpy().decode('utf-8'))

Portuguese:  e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
English:    and when you improve searchability , you actually take away the one advantage of print , which is serendipity .


In [5]:
train_en = train_examples.map(lambda pt, en: en)
train_pt = train_examples.map(lambda pt, en: pt)

## ボキャブラリーを生成する

データセットから*wordpiece*ボキャブラリーを生成する。すでにボキャブラリーのファイルがある場合にはこのステップは必要ない。

In [6]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [7]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [9]:
%%time
pt_vocab = bert_vocab.bert_vocab_from_dataset(
    train_pt.batch(1000).prefetch(2),
    **bert_vocab_args
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 1min 38s, sys: 2.54 s, total: 1min 40s
Wall time: 1min 35s


In [10]:
print(pt_vocab[:10])
print(pt_vocab[100:110])
print(pt_vocab[1000:1010])
print(pt_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['no', 'por', 'mais', 'na', 'eu', 'esta', 'muito', 'isso', 'isto', 'sao']
['90', 'desse', 'efeito', 'malaria', 'normalmente', 'palestra', 'recentemente', '##nca', 'bons', 'chave']
['##–', '##—', '##‘', '##’', '##“', '##”', '##⁄', '##€', '##♪', '##♫']


In [11]:
%%time
en_vocab = bert_vocab.bert_vocab_from_dataset(
    train_en.batch(1000).prefetch(2),
    **bert_vocab_args
)

CPU times: user 1min 13s, sys: 826 ms, total: 1min 14s
Wall time: 1min 13s


In [12]:
print(en_vocab[:10])
print(en_vocab[100:110])
print(en_vocab[1000:1010])
print(en_vocab[-10:])

['[PAD]', '[UNK]', '[START]', '[END]', '!', '#', '$', '%', '&', "'"]
['as', 'all', 'at', 'one', 'people', 're', 'like', 'if', 'our', 'from']
['choose', 'consider', 'extraordinary', 'focus', 'generation', 'killed', 'patterns', 'putting', 'scientific', 'wait']
['##_', '##`', '##ย', '##ร', '##อ', '##–', '##—', '##’', '##♪', '##♫']


ボキャブラリーファイルを作成する。

In [13]:
def write_vocab_file(filepath, vocab):
    with open(filepath, 'w') as f:
        for token in vocab:
            print(token, file=f)

In [14]:
write_vocab_file('pt_vacab.txt', pt_vocab)
write_vocab_file('en_vacab.txt', en_vocab)

In [15]:
ls *.txt

en_vacab.txt  pt_vacab.txt


## トークナイザの構築